In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import snowflake.connector
import ipywidgets as widgets
from IPython.display import display, clear_output

# -----------------------------
# Snowflake config
# -----------------------------
SNOWFLAKE_USER = "ABHRJ5665"
SNOWFLAKE_PASSWORD = "Abhrj@0000005665"
SNOWFLAKE_ACCOUNT = "XOCWKMU-NOC12545"
SNOWFLAKE_WAREHOUSE = "PIPELINE_WH"
SNOWFLAKE_DATABASE = "SHOP_DB"
SNOWFLAKE_SCHEMA = "PUBLIC"


def snowflake_conn():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )


Add Standardised ORDERDATE column , new column STD_ORDERDATE  to match with the date format of the dates table

In [9]:
conn = snowflake_conn()
cur = conn.cursor()

# Add STD_ORDERDATE if not exists
cur.execute("""
ALTER TABLE TRANSACTIONS
ADD COLUMN IF NOT EXISTS STD_ORDERDATE DATE;
""")

# Populate STD_ORDERDATE from ORDERDATE strings
cur.execute("""
UPDATE TRANSACTIONS
SET STD_ORDERDATE = TRY_TO_DATE(ORDERDATE, 'MM/DD/YYYY HH24:MI');
""")

# Quick check
cur.execute("SELECT ORDERDATE, STD_ORDERDATE FROM TRANSACTIONS LIMIT 5")
for row in cur.fetchall():
    print(row)

cur.close()
conn.close()


('2/24/2003 0:00', datetime.date(2003, 2, 24))
('5/7/2003 0:00', datetime.date(2003, 5, 7))
('7/1/2003 0:00', datetime.date(2003, 7, 1))
('8/25/2003 0:00', datetime.date(2003, 8, 25))
('10/10/2003 0:00', datetime.date(2003, 10, 10))


Analytics Q1-3

Provide the weekly, monthly and quarterly amount of sales
Provide  the weekly, monthly and quarterly total amount of refunds
Provide the total amount of items


In [2]:
# -----------------------------
# Pull transactions from Snowflake
# -----------------------------
conn = snowflake_conn()
query = """
SELECT 
    ORDERNUMBER,
    QUANTITYORDERED,
    TOTAL_AMOUNT,
    STD_ORDERDATE
FROM TRANSACTIONS
"""
df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Clean & convert types
# -----------------------------
df['ORDER_DATE'] = pd.to_datetime(df['STD_ORDERDATE'], errors='coerce')
df['QUANTITYORDERED'] = pd.to_numeric(df['QUANTITYORDERED'], errors='coerce')
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce')

# -----------------------------
# Compute metrics
# -----------------------------
df['SALES_AMOUNT'] = np.where(df['TOTAL_AMOUNT'] > 0, df['TOTAL_AMOUNT'], 0)
df['REFUND_AMOUNT'] = np.where(df['TOTAL_AMOUNT'] < 0, -df['TOTAL_AMOUNT'], 0)
df['TOTAL_ITEMS'] = df['QUANTITYORDERED'].fillna(0)

# -----------------------------
# Derive fiscal week/month/quarter
# -----------------------------
df['FISCAL_YEAR'] = df['ORDER_DATE'].dt.year
df['FISCAL_WEEK'] = df['ORDER_DATE'].dt.isocalendar().week
df['FISCAL_MONTH'] = df['ORDER_DATE'].dt.month
df['FISCAL_MONTH_NAME'] = df['ORDER_DATE'].dt.strftime('%b')
df['FISCAL_QUARTER'] = df['ORDER_DATE'].dt.quarter

df['WEEK_KEY'] = df['FISCAL_YEAR'].astype(str) + "-W" + df['FISCAL_WEEK'].astype(str)
df['MONTH_KEY'] = df['FISCAL_YEAR'].astype(str) + "-M" + df['FISCAL_MONTH'].astype(str)
df['QUARTER_KEY'] = df['FISCAL_YEAR'].astype(str) + "-Q" + df['FISCAL_QUARTER'].astype(str)

# -----------------------------
# Aggregation function
# -----------------------------
def aggregate_metrics(df, group_col):
    return df.groupby(group_col).agg(
        total_sales=('SALES_AMOUNT','sum'),
        total_refunds=('REFUND_AMOUNT','sum'),
        total_items=('TOTAL_ITEMS','sum'),
        orders=('ORDERNUMBER','nunique')
    ).reset_index()

# -----------------------------
# Multi-year interactive filter
# -----------------------------


year_min, year_max = int(df['FISCAL_YEAR'].min()), int(df['FISCAL_YEAR'].max())

year_range_slider = widgets.IntRangeSlider(
    value=[year_min, year_max],
    min=year_min,
    max=year_max,
    step=1,
    description='Year Range:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '600px'}
)

output = widgets.Output()

def update_dashboard(change):
    with output:
        clear_output()
        start_year, end_year = year_range_slider.value
        df_filtered = df[(df['FISCAL_YEAR'] >= start_year) & (df['FISCAL_YEAR'] <= end_year)]

        # Aggregations
        weekly_metrics = aggregate_metrics(df_filtered, 'WEEK_KEY')
        monthly_metrics = aggregate_metrics(df_filtered, 'MONTH_KEY')
        quarterly_metrics = aggregate_metrics(df_filtered, 'QUARTER_KEY')

        # Display tables
        print(f"📊 Weekly Metrics ({start_year}-{end_year})")
        display(weekly_metrics.head(10))
        print(f"📊 Monthly Metrics ({start_year}-{end_year})")
        display(monthly_metrics.head(10))
        print(f"📊 Quarterly Metrics ({start_year}-{end_year})")
        display(quarterly_metrics.head(10))

        # Charts
        fig_week = px.bar(weekly_metrics, x='WEEK_KEY', y='total_sales', title=f'Weekly Sales Amount ({start_year}-{end_year})')
        fig_week.show()

        fig_month = px.line(monthly_metrics, x='MONTH_KEY', y='total_refunds', title=f'Monthly Refund Amounts ({start_year}-{end_year})')
        fig_month.show()

        fig_quarter = px.bar(quarterly_metrics, x='QUARTER_KEY', y='total_items', title=f'Quarterly Total Items ({start_year}-{end_year})')
        fig_quarter.show()

year_range_slider.observe(update_dashboard, names='value')

display(year_range_slider, output)

# Initial dashboard display
update_dashboard(None)


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\2056638500.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


IntRangeSlider(value=(2003, 2005), continuous_update=False, description='Year Range:', layout=Layout(width='60…

Output()

Q4 Items’ Product Code is composed of a product family and a sub category, separated by an underscore. Provide the weekly, monthly and quarterly amount of sales for each product family.


In [3]:
# -----------------------------
# Pull PRODUCTCODE along with metrics
# -----------------------------
conn = snowflake_conn()
query = """
SELECT 
    ORDERNUMBER,
    QUANTITYORDERED,
    TOTAL_AMOUNT,
    STD_ORDERDATE,
    PRODUCTCODE
FROM TRANSACTIONS
"""
df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Clean & convert
# -----------------------------
df['ORDER_DATE'] = pd.to_datetime(df['STD_ORDERDATE'], errors='coerce')
df['QUANTITYORDERED'] = pd.to_numeric(df['QUANTITYORDERED'], errors='coerce')
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce')

df['SALES_AMOUNT'] = np.where(df['TOTAL_AMOUNT'] > 0, df['TOTAL_AMOUNT'], 0)
df['TOTAL_ITEMS'] = df['QUANTITYORDERED'].fillna(0)

# -----------------------------
# Split PRODUCTCODE
# -----------------------------
df[['PRODUCT_FAMILY', 'SUB_CATEGORY']] = df['PRODUCTCODE'].str.split('_', n=1, expand=True)

# -----------------------------
# Derive fiscal week/month/quarter
# -----------------------------
df['FISCAL_YEAR'] = df['ORDER_DATE'].dt.year
df['FISCAL_WEEK'] = df['ORDER_DATE'].dt.isocalendar().week
df['FISCAL_MONTH'] = df['ORDER_DATE'].dt.month
df['FISCAL_QUARTER'] = df['ORDER_DATE'].dt.quarter

df['WEEK_KEY'] = df['FISCAL_YEAR'].astype(str) + "-W" + df['FISCAL_WEEK'].astype(str)
df['MONTH_KEY'] = df['FISCAL_YEAR'].astype(str) + "-M" + df['FISCAL_MONTH'].astype(str)
df['QUARTER_KEY'] = df['FISCAL_YEAR'].astype(str) + "-Q" + df['FISCAL_QUARTER'].astype(str)

# -----------------------------
# Aggregation function by product family
# -----------------------------
def aggregate_by_family(df, group_col):
    return df.groupby([group_col, 'PRODUCT_FAMILY']).agg(
        total_sales=('SALES_AMOUNT','sum'),
        total_items=('TOTAL_ITEMS','sum'),
        orders=('ORDERNUMBER','nunique')
    ).reset_index()

# -----------------------------
# Multi-year interactive filter
# -----------------------------
year_min, year_max = int(df['FISCAL_YEAR'].min()), int(df['FISCAL_YEAR'].max())
year_range_slider = widgets.IntRangeSlider(
    value=[year_min, year_max],
    min=year_min,
    max=year_max,
    step=1,
    description='Year Range:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '600px'}
)
output = widgets.Output()

def update_dashboard(change):
    with output:
        clear_output()
        start_year, end_year = year_range_slider.value
        df_filtered = df[(df['FISCAL_YEAR'] >= start_year) & (df['FISCAL_YEAR'] <= end_year)]

        # Aggregations
        weekly_metrics = aggregate_by_family(df_filtered, 'WEEK_KEY')
        monthly_metrics = aggregate_by_family(df_filtered, 'MONTH_KEY')
        quarterly_metrics = aggregate_by_family(df_filtered, 'QUARTER_KEY')

        # Display tables
        print(f"📊 Weekly Metrics by Product Family ({start_year}-{end_year})")
        display(weekly_metrics.head(10))
        print(f"📊 Monthly Metrics by Product Family ({start_year}-{end_year})")
        display(monthly_metrics.head(10))
        print(f"📊 Quarterly Metrics by Product Family ({start_year}-{end_year})")
        display(quarterly_metrics.head(10))

        # Charts (example: top 5 families by sales)
        top_families = df_filtered['PRODUCT_FAMILY'].value_counts().head(5).index

        fig_week = px.bar(weekly_metrics[weekly_metrics['PRODUCT_FAMILY'].isin(top_families)],
                          x='WEEK_KEY', y='total_sales', color='PRODUCT_FAMILY',
                          title=f'Weekly Sales by Product Family ({start_year}-{end_year})')
        fig_week.show()

        fig_month = px.bar(monthly_metrics[monthly_metrics['PRODUCT_FAMILY'].isin(top_families)],
                           x='MONTH_KEY', y='total_sales', color='PRODUCT_FAMILY',
                           title=f'Monthly Sales by Product Family ({start_year}-{end_year})')
        fig_month.show()

        fig_quarter = px.bar(quarterly_metrics[quarterly_metrics['PRODUCT_FAMILY'].isin(top_families)],
                             x='QUARTER_KEY', y='total_sales', color='PRODUCT_FAMILY',
                             title=f'Quarterly Sales by Product Family ({start_year}-{end_year})')
        fig_quarter.show()

year_range_slider.observe(update_dashboard, names='value')

display(year_range_slider, output)

# Initial dashboard display
update_dashboard(None)


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\2375625193.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



IntRangeSlider(value=(2003, 2005), continuous_update=False, description='Year Range:', layout=Layout(width='60…

Output()

Q5 What are the best-selling and the second best-selling items in every region?

In [ ]:
conn = snowflake_conn()
query = """
SELECT 
    PRODUCTCODE,
    TOTAL_AMOUNT,
    TERRITORY
FROM TRANSACTIONS
"""
df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Clean & convert
# -----------------------------
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce')
df['SALES_AMOUNT'] = df['TOTAL_AMOUNT'].apply(lambda x: x if x > 0 else 0)  # Only count sales, ignore refunds

# -----------------------------
# Aggregate total sales by product per region
# -----------------------------
sales_by_region = df.groupby(['TERRITORY', 'PRODUCTCODE']).agg(
    total_sales=('SALES_AMOUNT','sum')
).reset_index()

# -----------------------------
# Rank products in each region
# -----------------------------
sales_by_region['rank'] = sales_by_region.groupby('TERRITORY')['total_sales'] \
                                         .rank(method='dense', ascending=False)

# -----------------------------
# Filter top 2 products per region
# -----------------------------
top2_products_region = sales_by_region[sales_by_region['rank'] <= 2].sort_values(['TERRITORY','rank'])

# -----------------------------
# Display table
# -----------------------------
print("🏆 Best-Selling and Second Best-Selling Items per Region")
display(top2_products_region[['TERRITORY', 'PRODUCTCODE', 'total_sales', 'rank']])

# -----------------------------
# Plot bar chart
# -----------------------------
fig_top2 = px.bar(
    top2_products_region,
    x='TERRITORY',
    y='total_sales',
    color='PRODUCTCODE',
    barmode='group',
    text='total_sales',
    title='🏆 Top 2 Best-Selling Items per Region'
)
fig_top2.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_top2.show()

C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\2273950412.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



IntRangeSlider(value=(2003, 2005), continuous_update=False, description='Year Range:', layout=Layout(width='60…

Output()

Q6 What is the difference between the revenue of each item and the revenue of the best-selling item in the same region?


In [12]:
# -----------------------------
# Pull relevant columns
# -----------------------------
conn = snowflake_conn()
query = """
SELECT 
    PRODUCTCODE,
    TOTAL_AMOUNT,
    TERRITORY
FROM TRANSACTIONS
"""
df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Clean & convert
# -----------------------------
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce')
df['SALES_AMOUNT'] = df['TOTAL_AMOUNT'].apply(lambda x: x if x > 0 else 0)

# -----------------------------
# Aggregate total sales per product per region
# -----------------------------
sales_by_region = df.groupby(['TERRITORY', 'PRODUCTCODE']).agg(
    total_sales=('SALES_AMOUNT','sum')
).reset_index()

# -----------------------------
# Find the best-selling revenue per region
# -----------------------------
best_sales = sales_by_region.groupby('TERRITORY')['total_sales'].max().reset_index()
best_sales.rename(columns={'total_sales': 'best_sales_in_region'}, inplace=True)

# -----------------------------
# Merge to compute difference
# -----------------------------
sales_diff = sales_by_region.merge(best_sales, on='TERRITORY')
sales_diff['revenue_diff'] = sales_diff['best_sales_in_region'] - sales_diff['total_sales']

# -----------------------------
# Display
# -----------------------------
print("📊 Revenue Difference from Best-Selling Item per Region")
display(sales_diff.sort_values(['TERRITORY','revenue_diff']))

fig_diff = px.bar(sales_diff,
                  x='TERRITORY',
                  y='revenue_diff',
                  color='PRODUCTCODE',
                  title='📊 Revenue Difference from Best-Selling Item per Region',
                  text='revenue_diff')
fig_diff.show()



C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\3657323722.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



📊 Revenue Difference from Best-Selling Item per Region


,TERRITORY,PRODUCTCODE,total_sales,best_sales_in_region,revenue_diff
48,APAC,S24_1937,4363,4363,0
38,APAC,S18_4409,4164,4363,199
28,APAC,S18_2957,3943,4363,420
32,APAC,S18_3320,3817,4363,546
61,APAC,S24_3432,3306,4363,1057
...,...,...,...,...,...
270,Japan,S700_2610,0,4729,4729
272,Japan,S700_2834,0,4729,4729
273,Japan,S700_3167,0,4729,4729
274,Japan,S700_3962,0,4729,4729


Q7 Refunds are subject to a restocking fee of 10%. Write an UDF that calculates the fee and use it to calculate the final amount of refund.

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# -----------------------------
# Pull transactions from Snowflake
# -----------------------------
conn = snowflake_conn()
query = """
SELECT 
    ORDERNUMBER,
    TOTAL_AMOUNT,
    FINAL_REFUND_AMOUNT,
    STD_ORDERDATE
FROM TRANSACTIONS
"""
df = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Clean & convert
# -----------------------------
df['ORDER_DATE'] = pd.to_datetime(df['STD_ORDERDATE'], errors='coerce')
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce')
df['FINAL_REFUND_AMOUNT'] = pd.to_numeric(df['FINAL_REFUND_AMOUNT'], errors='coerce')

# -----------------------------
# Only keep refund rows
# -----------------------------
df_refunds = df[df['FINAL_REFUND_AMOUNT'] > 0].copy()

# -----------------------------
# Calculate intermediate metrics
# -----------------------------
df_refunds['REFUND_AMOUNT'] = np.where(df_refunds['TOTAL_AMOUNT'] < 0, -df_refunds['TOTAL_AMOUNT'], 0)
df_refunds['RESTOCK_FEE'] = df_refunds['REFUND_AMOUNT'] - df_refunds['FINAL_REFUND_AMOUNT']

# -----------------------------
# Derive fiscal week/month/quarter
# -----------------------------
df_refunds['FISCAL_YEAR'] = df_refunds['ORDER_DATE'].dt.year
df_refunds['FISCAL_WEEK'] = df_refunds['ORDER_DATE'].dt.isocalendar().week
df_refunds['FISCAL_MONTH'] = df_refunds['ORDER_DATE'].dt.month
df_refunds['FISCAL_QUARTER'] = df_refunds['ORDER_DATE'].dt.quarter

df_refunds['WEEK_KEY'] = df_refunds['FISCAL_YEAR'].astype(str) + "-W" + df_refunds['FISCAL_WEEK'].astype(str)
df_refunds['MONTH_KEY'] = df_refunds['FISCAL_YEAR'].astype(str) + "-M" + df_refunds['FISCAL_MONTH'].astype(str)
df_refunds['QUARTER_KEY'] = df_refunds['FISCAL_YEAR'].astype(str) + "-Q" + df_refunds['FISCAL_QUARTER'].astype(str)

# -----------------------------
# Aggregation function
# -----------------------------
def aggregate_refunds(df, group_col):
    return df.groupby(group_col).agg(
        total_refunds=('REFUND_AMOUNT','sum'),       # Before restocking fee
        total_restocks=('RESTOCK_FEE','sum'),        # Restocking fee
        final_refunds=('FINAL_REFUND_AMOUNT','sum'), # After restocking fee
        num_refunds=('ORDERNUMBER','nunique')        # Count of refund orders
    ).reset_index()

# -----------------------------
# Multi-year filter widget
# -----------------------------
year_min, year_max = int(df_refunds['FISCAL_YEAR'].min()), int(df_refunds['FISCAL_YEAR'].max())
year_range_slider = widgets.IntRangeSlider(
    value=[year_min, year_max],
    min=year_min,
    max=year_max,
    step=1,
    description='Year Range:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout={'width': '600px'}
)

output = widgets.Output()

def update_refund_dashboard(change):
    with output:
        clear_output()
        start_year, end_year = year_range_slider.value
        df_filtered = df_refunds[(df_refunds['FISCAL_YEAR'] >= start_year) & (df_refunds['FISCAL_YEAR'] <= end_year)]

        # Aggregations
        weekly_refunds = aggregate_refunds(df_filtered, 'WEEK_KEY')
        monthly_refunds = aggregate_refunds(df_filtered, 'MONTH_KEY')
        quarterly_refunds = aggregate_refunds(df_filtered, 'QUARTER_KEY')

        # Display tables
        print(f"📊 Weekly Refund Metrics ({start_year}-{end_year})")
        display(weekly_refunds.head(10))
        print(f"📊 Monthly Refund Metrics ({start_year}-{end_year})")
        display(monthly_refunds.head(10))
        print(f"📊 Quarterly Refund Metrics ({start_year}-{end_year})")
        display(quarterly_refunds.head(10))

        # Charts
        fig_week = px.bar(weekly_refunds, x='WEEK_KEY', y='final_refunds', 
                          title=f'Weekly Refunds after Restocking Fee ({start_year}-{end_year})',
                          labels={'final_refunds':'Final Refund Amount'})
        fig_week.show()

        fig_month = px.line(monthly_refunds, x='MONTH_KEY', y='final_refunds', 
                            title=f'Monthly Refunds after Restocking Fee ({start_year}-{end_year})',
                            labels={'final_refunds':'Final Refund Amount'})
        fig_month.show()

        fig_quarter = px.bar(quarterly_refunds, x='QUARTER_KEY', y='final_refunds', 
                             title=f'Quarterly Refunds after Restocking Fee ({start_year}-{end_year})',
                             labels={'final_refunds':'Final Refund Amount'})
        fig_quarter.show()

year_range_slider.observe(update_refund_dashboard, names='value')

display(year_range_slider, output)

# Initial display
update_refund_dashboard(None)


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\2273950412.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



IntRangeSlider(value=(2003, 2005), continuous_update=False, description='Year Range:', layout=Layout(width='60…

Output()

Q8 Segment the customers into High Value, Medium Value, and Low Value  customers based on net spending and order frequency. Show the top 10 spending customers and their details so they can be contacted

In [23]:
import pandas as pd
import plotly.express as px

# -----------------------------
# Pull aggregated customer data from Snowflake
# -----------------------------
conn = snowflake_conn()
query = """
SELECT 
    CUSTOMERNAME,
    COUNT(ORDERNUMBER) AS NUM_ORDERS,
    SUM(TOTAL_AMOUNT) AS TOTAL_SPENT,
    SUM(FINAL_REFUND_AMOUNT) AS TOTAL_REFUNDS,
    SUM(TOTAL_AMOUNT) - SUM(FINAL_REFUND_AMOUNT) AS NET_SPENDING,
    MAX(STD_ORDERDATE) AS LAST_ORDER_DATE
FROM TRANSACTIONS
GROUP BY CUSTOMERNAME
"""
df_customers = pd.read_sql(query, conn)
conn.close()

# -----------------------------
# Top 10 by Net Spending (show order frequency in brackets)
# -----------------------------
df_top10_spent = df_customers.sort_values(by='NET_SPENDING', ascending=False).head(10)
df_top10_spent['CUSTOMER_LABEL'] = df_top10_spent.apply(
    lambda row: f"{row['CUSTOMERNAME']} ({row['NUM_ORDERS']} orders)", axis=1
)

# Horizontal bar chart
fig_top_spent = px.bar(
    df_top10_spent[::-1],  # Reverse to show highest on top
    x='NET_SPENDING',
    y='CUSTOMER_LABEL',
    orientation='h',
    text='NET_SPENDING',
    color='NET_SPENDING',
    title='Top 10 Customers by Net Spending (with Order Frequency)',
    labels={'NET_SPENDING':'Net Spending ($)', 'CUSTOMER_LABEL':'Customer (Orders)'}
)
fig_top_spent.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_top_spent.show()

print("📊 Top 10 Customers by Net Spending")
display(df_top10_spent[['CUSTOMERNAME', 'NUM_ORDERS', 'TOTAL_SPENT', 'TOTAL_REFUNDS', 'NET_SPENDING', 'LAST_ORDER_DATE']])

# -----------------------------
# Top 10 by Order Frequency (show net spending in brackets)
# -----------------------------
df_top10_orders = df_customers.sort_values(by='NUM_ORDERS', ascending=False).head(10)
df_top10_orders['CUSTOMER_LABEL'] = df_top10_orders.apply(
    lambda row: f"{row['CUSTOMERNAME']} (${row['NET_SPENDING']:.2f})", axis=1
)

# Horizontal bar chart
fig_top_orders = px.bar(
    df_top10_orders[::-1],  # Reverse to show highest on top
    x='NUM_ORDERS',
    y='CUSTOMER_LABEL',
    orientation='h',
    text='NUM_ORDERS',
    color='NUM_ORDERS',
    title='Top 10 Customers by Order Frequency (with Net Spending)',
    labels={'NUM_ORDERS':'Number of Orders', 'CUSTOMER_LABEL':'Customer (Net Spending)'}
)
fig_top_orders.update_traces(texttemplate='%{text}', textposition='outside')
fig_top_orders.show()

print("📊 Top 10 Customers by Order Frequency")
display(df_top10_orders[['CUSTOMERNAME', 'NUM_ORDERS', 'TOTAL_SPENT', 'TOTAL_REFUNDS', 'NET_SPENDING', 'LAST_ORDER_DATE']])


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\4084370279.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



📊 Top 10 Customers by Net Spending


,CUSTOMERNAME,NUM_ORDERS,TOTAL_SPENT,TOTAL_REFUNDS,NET_SPENDING,LAST_ORDER_DATE
88,Signal Gift Stores,29,12585.0,8828.1,3756.9,2004-11-29
86,"Iberia Gift Imports, Corp.",15,7565.0,4790.7,2774.3,2004-10-06
33,Super Scale Inc.,17,5429.0,4608.0,821.0,2004-05-04
78,Technics Stores Inc.,34,11322.0,10745.1,576.9,2005-01-05
53,Classic Legends Inc.,20,6118.0,5544.0,574.0,2004-11-21
45,Microscale Inc.,10,3119.0,2655.0,464.0,2004-11-03
12,Souveniers And Things Co.,46,14644.0,14944.5,-300.5,2005-05-29
85,Online Mini Collectables,15,5082.0,5733.9,-651.9,2004-09-10
65,giftsbymail.co.uk,26,7410.0,9847.8,-2437.8,2004-11-01
48,Toys4GrownUps.com,30,6883.0,9534.6,-2651.6,2005-01-12


📊 Top 10 Customers by Order Frequency


,CUSTOMERNAME,NUM_ORDERS,TOTAL_SPENT,TOTAL_REFUNDS,NET_SPENDING,LAST_ORDER_DATE
14,Euro Shopping Channel,259,-8564.0,125802.0,-134366.0,2005-05-31
20,Mini Gifts Distributors Ltd.,180,22668.0,66629.7,-43961.7,2005-05-29
59,"Australian Collectors, Co.",55,-2142.0,23238.9,-25380.9,2004-11-29
7,La Rochelle Gifts,53,1778.0,20412.9,-18634.9,2005-05-31
83,"AV Stores, Co.",51,8870.0,16042.5,-7172.5,2004-11-17
0,Land of Toys Inc.,49,7152.0,20531.7,-13379.7,2004-11-15
28,Muscle Machine Inc,48,-7027.0,22693.5,-29720.5,2004-12-01
91,Rovelli Gifts,48,2494.0,20547.9,-18053.9,2004-11-12
12,Souveniers And Things Co.,46,14644.0,14944.5,-300.5,2005-05-29
37,"Anna's Decorations, Ltd",46,-2907.0,21202.2,-24109.2,2005-03-09


Q9 Calculate the Aggregate on Fiscal Dates


In [28]:
import pandas as pd
import plotly.express as px
import numpy as np

# -----------------------------
# Pull transactions and fiscal dates from Snowflake
# -----------------------------
conn = snowflake_conn()
df_txn = pd.read_sql("""
    SELECT ORDERNUMBER, TOTAL_AMOUNT, FINAL_REFUND_AMOUNT, STD_ORDERDATE
    FROM TRANSACTIONS
""", conn)

df_dates = pd.read_sql("""
    SELECT CALENDAR_DATE, FISCAL_YEAR, FISCAL_QUARTER, FISCAL_MONTH_NUMBER,
           FISCAL_FIRST_DAY_OF_WEEK, FISCAL_LAST_DAY_OF_WEEK,
           FISCAL_FIRST_DAY_OF_MONTH, FISCAL_LAST_DAY_OF_MONTH,
           FISCAL_YEAR_WEEK, FISCAL_YEAR_MONTH, FISCAL_YEAR_QUARTER
    FROM DATES
""", conn)
conn.close()

# -----------------------------
# Convert dates
# -----------------------------
df_txn['STD_ORDERDATE'] = pd.to_datetime(df_txn['STD_ORDERDATE'], errors='coerce')
df_txn = df_txn.dropna(subset=['STD_ORDERDATE'])

df_dates['CALENDAR_DATE'] = pd.to_datetime(df_dates['CALENDAR_DATE'], errors='coerce')
df_dates = df_dates.dropna(subset=['CALENDAR_DATE'])

# Ensure fiscal period columns are datetime
for col in ['FISCAL_FIRST_DAY_OF_WEEK', 'FISCAL_LAST_DAY_OF_WEEK',
            'FISCAL_FIRST_DAY_OF_MONTH', 'FISCAL_LAST_DAY_OF_MONTH']:
    df_dates[col] = pd.to_datetime(df_dates[col], errors='coerce')

# -----------------------------
# Merge transactions with fiscal dates
# -----------------------------
df = df_txn.merge(df_dates, left_on='STD_ORDERDATE', right_on='CALENDAR_DATE', how='left')

# -----------------------------
# Handle any missing fiscal dates
# -----------------------------
# For transactions without matching fiscal dates, approximate using the transaction date itself
df['FISCAL_YEAR'] = df['FISCAL_YEAR'].fillna(df['STD_ORDERDATE'].dt.year)
df['FISCAL_QUARTER'] = df['FISCAL_QUARTER'].fillna(df['STD_ORDERDATE'].dt.quarter)
df['FISCAL_MONTH_NUMBER'] = df['FISCAL_MONTH_NUMBER'].fillna(df['STD_ORDERDATE'].dt.month)
df['FISCAL_YEAR_WEEK'] = df['FISCAL_YEAR_WEEK'].fillna(df['STD_ORDERDATE'].dt.isocalendar().week)
df['FISCAL_YEAR_MONTH'] = df['FISCAL_YEAR_MONTH'].fillna(df['STD_ORDERDATE'].dt.month)
df['FISCAL_YEAR_QUARTER'] = df['FISCAL_YEAR_QUARTER'].fillna(df['STD_ORDERDATE'].dt.quarter)

# -----------------------------
# Clean numeric columns
# -----------------------------
df['TOTAL_AMOUNT'] = pd.to_numeric(df['TOTAL_AMOUNT'], errors='coerce').fillna(0)
df['FINAL_REFUND_AMOUNT'] = pd.to_numeric(df['FINAL_REFUND_AMOUNT'], errors='coerce').fillna(0)
df['NET_REVENUE'] = df['TOTAL_AMOUNT'] - df['FINAL_REFUND_AMOUNT']

# -----------------------------
# Generate fiscal period keys
# -----------------------------
df['WEEK_KEY'] = df['FISCAL_YEAR'].astype(int).astype(str) + "-W" + df['FISCAL_YEAR_WEEK'].astype(int).astype(str)
df['MONTH_KEY'] = df['FISCAL_YEAR'].astype(int).astype(str) + "-M" + df['FISCAL_YEAR_MONTH'].astype(int).astype(str)
df['QUARTER_KEY'] = df['FISCAL_YEAR'].astype(int).astype(str) + "-Q" + df['FISCAL_YEAR_QUARTER'].astype(int).astype(str)

# -----------------------------
# Aggregation function
# -----------------------------
def aggregate_metrics(df, group_col):
    return df.groupby(group_col).agg(
        total_sales=('TOTAL_AMOUNT','sum'),
        total_refunds=('FINAL_REFUND_AMOUNT','sum'),
        net_revenue=('NET_REVENUE','sum'),
        total_items=('ORDERNUMBER','count')
    ).reset_index()

# -----------------------------
# Aggregate by fiscal week, month, quarter
# -----------------------------
weekly_agg = aggregate_metrics(df, 'WEEK_KEY')
monthly_agg = aggregate_metrics(df, 'MONTH_KEY')
quarterly_agg = aggregate_metrics(df, 'QUARTER_KEY')

# -----------------------------
# Display tables
# -----------------------------
print("📊 Weekly Aggregates")
display(weekly_agg.head(10))

print("📊 Monthly Aggregates")
display(monthly_agg.head(10))

print("📊 Quarterly Aggregates")
display(quarterly_agg.head(10))

# -----------------------------
# Visualizations
# -----------------------------
px.bar(weekly_agg, x='WEEK_KEY', y='net_revenue', title='Weekly Net Revenue (Fiscal)').show()
px.bar(monthly_agg, x='MONTH_KEY', y='net_revenue', title='Monthly Net Revenue (Fiscal)').show()
px.bar(quarterly_agg, x='QUARTER_KEY', y='net_revenue', title='Quarterly Net Revenue (Fiscal)').show()


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\106385294.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\106385294.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



📊 Weekly Aggregates


C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\106385294.py:46: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\106385294.py:47: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\Abhi0618\AppData\Local\Temp\ipykernel_13280\106385294.py:48: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', 

,WEEK_KEY,total_sales,total_refunds,net_revenue,total_items
0,2003-W10,-4033,8832.6,-12865.6,16
1,2003-W11,-3451,3809.7,-7260.7,6
2,2003-W12,1765,5559.3,-3794.3,16
3,2003-W13,1657,6500.7,-4843.7,12
4,2003-W14,-5096,9581.4,-14677.4,15
5,2003-W15,-308,277.2,-585.2,1
6,2003-W16,6358,4502.7,1855.3,12
7,2003-W17,-1334,1200.6,-2534.6,1
8,2003-W18,-10426,17823.6,-28249.6,29
9,2003-W19,-3429,13041.0,-16470.0,22


📊 Monthly Aggregates


,MONTH_KEY,total_sales,total_refunds,net_revenue,total_items
0,2003-M1,-5262,21039.3,-26301.3,39
1,2003-M10,-49,77292.0,-77341.0,158
2,2003-M11,-9493,139755.6,-149248.6,296
3,2003-M12,-10978,34258.5,-45236.5,70
4,2003-M2,14220,14669.1,-449.1,41
5,2003-M3,-4062,24702.3,-28764.3,50
6,2003-M4,-10806,33385.5,-44191.5,58
7,2003-M5,-1840,32364.0,-34204.0,58
8,2003-M6,972,18628.2,-17656.2,46
9,2003-M7,13105,15912.0,-2807.0,50


📊 Quarterly Aggregates


,QUARTER_KEY,total_sales,total_refunds,net_revenue,total_items
0,2003-Q1,4896,60410.7,-55514.7,130
1,2003-Q2,-11674,84377.7,-96051.7,162
2,2003-Q3,24314,65600.1,-41286.1,184
3,2003-Q4,-20520,251306.1,-271826.1,524
4,2004-Q1,-12089,110581.2,-122670.2,233
5,2004-Q2,-10525,105187.5,-115712.5,223
6,2004-Q3,-7161,150656.4,-157817.4,319
7,2004-Q4,16726,235431.9,-218705.9,570
8,2005-Q1,-20010,147341.7,-167351.7,302
9,2005-Q2,-1519,78832.8,-80351.8,176
